In [1]:
from langchain.agents import create_sql_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase

llm = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-3.5-turbo-1106",
)
db = SQLDatabase.from_uri("sqlite:///movies.sqlite")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

agent.invoke("Give me the 5 directors that have the highest grossing films.")



> Entering new AgentExecutor chain...


Authentication failed for https://api.smith.langchain.com/runs. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs', '{"detail":"Using legacy API key. Please generate a new API key."}')


Action: sql_db_list_tables
Action Input: 
Observation: directors, movies
Thought:

Authentication failed for https://api.smith.langchain.com/runs/28ad973d-dc3d-408e-b306-31f46994d871. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/28ad973d-dc3d-408e-b306-31f46994d871', '{"detail":"Using legacy API key. Please generate a new API key."}')


I should now query the schema of the directors and movies tables to see what columns I can use to find the highest grossing films.
Action: sql_db_schema
Action Input: 'directors, movies'
Observation: Error: table_names {"'directors", "movies'"} not found in database
Thought:I made a mistake in the input for the sql_db_schema tool. I need to remove the single quotes around the table names.
Action: sql_db_schema
Action Input: directors, movies
Observation: 
CREATE TABLE directors (
	name TEXT, 
	id INTEGER, 
	gender INTEGER, 
	uid INTEGER, 
	department TEXT, 
	PRIMARY KEY (id)
)

/*
3 rows from directors table:
name	id	gender	uid	department
James Cameron	4762	2	2710	Directing
Gore Verbinski	4763	2	1704	Directing
Sam Mendes	4764	2	39	Directing
*/


CREATE TABLE movies (
	id INTEGER, 
	original_title VARCHAR, 
	budget INTEGER, 
	popularity INTEGER, 
	release_date TEXT, 
	revenue INTEGER, 
	title TEXT, 
	vote_average REAL, 
	vote_count INTEGER, 
	overview TEXT, 
	tagline TEXT, 
	uid INTEGER

{'input': 'Give me the 5 directors that have the highest grossing films.',
 'output': 'The 5 directors with the highest grossing films are director_id 4799 with $9,147,393,164, director_id 4777 with $6,498,642,820, director_id 4762 with $5,883,569,439, director_id 4788 with $5,832,524,638, and director_id 4765 with $4,227,483,234.'}